In [2]:
import librosa
import librosa.display
import time
import matplotlib.pyplot as plt
import numpy as np
import torchlibrosa as tl
import torch
import pandas as pd
from torch.utils.data import Dataset
from torch.utils.data import DataLoader

In [3]:
DATA_DIR = "./MUSIC_DATA/maestro-v3.0.0"
df = pd.read_csv(f"{DATA_DIR}/maestro-v3.0.0.csv")
df

,canonical_composer,canonical_title,split,year,midi_filename,audio_filename,duration
0,Alban Berg,Sonata Op. 1,train,2018,2018/MIDI-Unprocessed_Chamber3_MID--AUDIO_10_R...,2018/MIDI-Unprocessed_Chamber3_MID--AUDIO_10_R...,698.661160
1,Alban Berg,Sonata Op. 1,train,2008,2008/MIDI-Unprocessed_03_R2_2008_01-03_ORIG_MI...,2008/MIDI-Unprocessed_03_R2_2008_01-03_ORIG_MI...,759.518471
2,Alban Berg,Sonata Op. 1,train,2017,2017/MIDI-Unprocessed_066_PIANO066_MID--AUDIO-...,2017/MIDI-Unprocessed_066_PIANO066_MID--AUDIO-...,464.649433
3,Alexander Scriabin,"24 Preludes Op. 11, No. 13-24",train,2004,2004/MIDI-Unprocessed_XP_21_R1_2004_01_ORIG_MI...,2004/MIDI-Unprocessed_XP_21_R1_2004_01_ORIG_MI...,872.640588
4,Alexander Scriabin,"3 Etudes, Op. 65",validation,2006,2006/MIDI-Unprocessed_17_R1_2006_01-06_ORIG_MI...,2006/MIDI-Unprocessed_17_R1_2006_01-06_ORIG_MI...,397.857508
...,...,...,...,...,...,...,...
1271,Wolfgang Amadeus Mozart,"Sonata in F Major, K280",test,2004,2004/MIDI-Unprocessed_XP_14_R1_2004_04_ORIG_MI...,2004/MIDI-Unprocessed_XP_14_R1_2004_04_ORIG_MI...,241.470442
1272,Wolfgang Amadeus Mozart,"Sonata in F Major, K280",train,2004,2004/MIDI-Unprocessed_XP_14_R1_2004_04_ORIG_MI...,2004/MIDI-Unprocessed_XP_14_R1_2004_04_ORIG_MI...,114.696243
1273,Wolfgang Amadeus Mozart,"Sonata in F Major, K533",validation,2004,2004/MIDI-Unprocessed_SMF_12_01_2004_01-05_ORI...,2004/MIDI-Unprocessed_SMF_12_01_2004_01-05_ORI...,1139.198478
1274,Wolfgang Amadeus Mozart,"Sonata in F Major, K533/K494",validation,2018,2018/MIDI-Unprocessed_Recital17-19_MID--AUDIO_...,2018/MIDI-Unprocessed_Recital17-19_MID--AUDIO_...,1068.751602


In [4]:
selection = ["audio_filename", "duration"]  # the only features we care about
train_entries = np.array(df[df.split == "train"][selection])
valid_entries = np.array(df[df.split == "validation"][selection])
test_entries = np.array(df[df.split == "test"][selection])
valid_entries

array([['2006/MIDI-Unprocessed_17_R1_2006_01-06_ORIG_MID--AUDIO_17_R1_2006_04_Track04_wav.wav',
        397.857508125],
       ['2009/MIDI-Unprocessed_07_R1_2009_04-05_ORIG_MID--AUDIO_07_R1_2009_07_R1_2009_04_WAV.wav',
        400.557825938],
       ['2009/MIDI-Unprocessed_02_R1_2009_03-06_ORIG_MID--AUDIO_02_R1_2009_02_R1_2009_05_WAV.wav',
        167.085837188],
       ['2009/MIDI-Unprocessed_11_R1_2009_06-09_ORIG_MID--AUDIO_11_R1_2009_11_R1_2009_06_WAV.wav',
        138.421335938],
       ['2013/ORIG-MIDI_01_7_6_13_Group__MID--AUDIO_03_R1_2013_wav--4.wav',
        440.110403906],
       ['2014/MIDI-UNPROCESSED_14-15_R1_2014_MID--AUDIO_15_R1_2014_wav--3.wav',
        365.655350156],
       ['2014/MIDI-UNPROCESSED_14-15_R1_2014_MID--AUDIO_15_R1_2014_wav--4.wav',
        319.910203594],
       ['2006/MIDI-Unprocessed_22_R1_2006_01-04_ORIG_MID--AUDIO_22_R1_2006_02_Track02_wav.wav',
        1188.161205],
       ['2018/MIDI-Unprocessed_Recital8_MID--AUDIO_08_R1_2018_wav--2.wav',
        10

In [6]:
class MusicDataset(Dataset):
    def __init__(self, annotations, win_size, sr):
        self.annotations = annotations
        self.win_sz = win_size  # length of each sample
        self.sr = sr
        self.mel_extractor = \
            torch.nn.Sequential( # weird design choice
                tl.Spectrogram(
                    hop_length=hop_length,
                    win_length=win_length,
                ), tl.LogmelFilterBank(
                    sr=sample_rate,
                    n_mels=n_mels,
                    is_log=False, # Default is true
                ))  
    
    def __len__(self):
        return len(self.annotations)
    
    def __getitem__(self, idx):
        length = int(np.ceil(self.annotations[idx,1]))
        path = f"{DATA_DIR}/{self.annotations[idx,0]}"
        frames = np.array([librosa.load(path, offset=t, duration=self.win_sz, sr=self.sr)[0] 
                      for t in range(0,length,self.win_sz)])
        return frames
    
sample_rate = 44100
win_length = 2048
hop_length = 512
n_mels = 128

# TorchLibrosa feature extractor the same as librosa.feature.melspectrogram()


In [7]:
raw = MusicDataset(valid_entries,20,44100)
loader = DataLoader(raw,batch_size=None,batch_sampler=None,shuffle=False,num_workers=0)

/opt/conda/lib/python3.7/site-packages/torchlibrosa/stft.py:193: FutureWarning: Pass size=2048 as keyword args. From version 0.10 passing these as positional arguments will result in an error
  fft_window = librosa.util.pad_center(fft_window, n_fft)


In [8]:
x = raw.mel_extractor(next(iter(loader))).squeeze()

In [ ]:
next(iter(loader)).shape